## test9.ipynb

Compare a ei_network simulation against an equivalent ei_net simulation.

By Sergio Verduzco
January 2018

In [1]:
%cd ..
#import ei_network
from ei_network import *
from ei_net import *

/home/z/projects/draculab


In [2]:
# Create ei_network network
layer_names = ['LD', 'LU', 'RD', 'RU'] # left down, left up, right down, right up
net1 = ei_network(layer_names)

In [3]:
# Configure layer parameters
LD = net1.layers['LD']
LU = net1.layers['LU']
RD = net1.layers['RD']
RU = net1.layers['RU']

centers = ( [-0.5,-0.5], [-0.5, 0.5], [0.5,-0.5], [0.5, 0.5] )
layers = [LD, LU, RD, RU]
for lyr, cntr in zip(layers, centers):
    for geom in ['e_geom', 'i_geom', 'x_geom']:
        lyr.set_param(geom, 'center', cntr)
        lyr.set_param(geom, 'extent', [1., 1.])
        lyr.set_param(geom, 'jitter', 0.)
        lyr.set_param(geom, 'rows', 4)
        lyr.set_param(geom, 'columns', 4)
    # A single input unit per layer, located at the center of the layer
    lyr.set_param('x_geom', 'rows', 1)
    lyr.set_param('x_geom', 'columns', 1)
    

In [4]:
# Add interlayer connections
pops = ['e', 'i']
# We'll allow any two populations to connect
for source in layer_names:
    for target in layer_names:
        if source != target:
            for pop1 in pops:
                for pop2 in pops:
                    net1.add_connection((source, pop1), (target, pop2))

In [6]:
# Configure interlayer connections
# I don't need to setup any interlayer transformations, just avoid periodic boundaries
for source in layer_names:
    for target in layer_names:
        if source != target:
            for pop1 in pops:
                for pop2 in pops:
                    net1.set_param(source+pop1+'_'+target+pop2+'_conn', 'edge_wrap', False )

In [7]:
net1.build()

Building layer RD
Building layer RU
Building layer LU
Building layer LD
Creating LDe_LUe connection
Creating LDe_LUi connection
Creating LDi_LUe connection
Creating LDi_LUi connection
Creating LDe_RDe connection
Creating LDe_RDi connection
Creating LDi_RDe connection
Creating LDi_RDi connection
Creating LDe_RUe connection
Creating LDe_RUi connection
Creating LDi_RUe connection
Creating LDi_RUi connection
Creating LUe_LDe connection
Creating LUe_LDi connection
Creating LUi_LDe connection
Creating LUi_LDi connection
Creating LUe_RDe connection
Creating LUe_RDi connection
Creating LUi_RDe connection
Creating LUi_RDi connection
Creating LUe_RUe connection
Creating LUe_RUi connection
Creating LUi_RUe connection
Creating LUi_RUi connection
Creating RDe_LDe connection
Creating RDe_LDi connection
Creating RDi_LDe connection
Creating RDi_LDi connection
Creating RDe_LUe connection
Creating RDe_LUi connection
Creating RDi_LUe connection
Creating RDi_LUi connection
Creating RDe_RUe connection
Crea

/home/z/projects/draculab/topology.py:390: UserWarning: Zero connections created with topo_connect
  warn('Zero connections created with topo_connect', UserWarning)


In [ ]:
ei_net.run(5, 1.)

In [ ]:
%matplotlib inline
ei_net.basic_plot('S')

In [ ]:
ei_net.basic_plot('M')

In [ ]:
ei_net.double_anim(M.e+S.e+S.x, thr=0.45)

In [ ]:
# Plot the highest unit activities
%matplotlib inline
n_top = 4  # how many units to plot
top_pop = np.array(S.e)  # population to visualize
top_units = ei_net.all_activs[top_pop].sum(1).argpartition(-n_top)[-n_top:]
print(top_units)
top_act_fig = plt.figure(figsize=(18,8))
top_acts = ei_net.all_activs[top_pop[top_units]].transpose()
plt.plot(ei_net.all_times, top_acts)
plt.show()

In [ ]:
# Plot the lowest unit activities
n_bottom = 4  # how many units to plot
bottom_pop = np.array(M.e)  # population to visualize
bottom_units = ei_net.all_activs[bottom_pop].sum(1).argpartition(n_bottom)[0:n_bottom-1]
print(bottom_units)
bottom_act_fig = plt.figure(figsize=(18,8))
bottom_acts = ei_net.all_activs[bottom_pop[bottom_units]].transpose()
plt.plot(ei_net.all_times, bottom_acts)
plt.show()

In [8]:
net1.conn_anim(LD.e, LD.e, slider=True)

interactive(children=(IntSlider(value=8, description='frame', max=16, min=1), Output()), _dom_classes=('widget-interact',))

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>>

In [ ]:
ei_net.act_anim(S.e+M.e, 0.2)

In [ ]:
ei_net.hist_anim(S.e+M.e)

In [ ]:
# logging
ei_net.log()